In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in incidence csv from github

mortality_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Age_Mortality_Cancer_DB.csv")                         
#get rid of commas in columns
mortality_df = mortality_df.replace(',','', regex=True)
mortality_df.head()     

,average_annual_count,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,abbrv,median_household_income_2015,median_household_income_2016,median_household_income_2017,median_household_income_2018,median_household_income_2019
0,4,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,38.8,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
1,48,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,50+,606.5,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
2,18,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<65,84.7,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
3,35,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,65+,963.0,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
4,9,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,30.2,Jackson County AL,53.0,52195,51988,51828,51621,51626,AL,40201.0,41407.0,42658.0,41929.0,44322.0


In [3]:
#check dtypes
mortality_df.dtypes

average_annual_count                     int64
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
abbrv                                   object
median_household_income_2015           float64
median_household_income_2016           float64
median_household_income_2017           float64
median_household_income_2018           float64
median_household_income_2019           float64
dtype: object

In [4]:
#get count of rows
mortality_df.count()

average_annual_count                   8170
primary_state_name                     8170
cancer                                 8170
race_ethnicity                         8170
sex                                    8170
age                                    8170
age_adjusted_rate                      8170
county_state                           8170
index_of_medical_underservice_score    7091
popestimate2015                        8170
popestimate2016                        8170
popestimate2017                        8170
popestimate2018                        8170
popestimate2019                        8170
abbrv                                  8152
median_household_income_2015           8152
median_household_income_2016           8152
median_household_income_2017           8152
median_household_income_2018           8152
median_household_income_2019           8152
dtype: int64

In [5]:
#get count of values for age
mortality_df["age"].value_counts()

50+    2420
65+    2406
<65    2198
<50    1146
Name: age, dtype: int64

In [6]:
#get count of NA values for MUA
mortality_df["index_of_medical_underservice_score"].isna().sum()

1079

In [7]:
#drop rows where MUA index is null
mortality_df = mortality_df[mortality_df["index_of_medical_underservice_score"].isna() == False]

In [8]:
#get count of rows
mortality_df.count()

average_annual_count                   7091
primary_state_name                     7091
cancer                                 7091
race_ethnicity                         7091
sex                                    7091
age                                    7091
age_adjusted_rate                      7091
county_state                           7091
index_of_medical_underservice_score    7091
popestimate2015                        7091
popestimate2016                        7091
popestimate2017                        7091
popestimate2018                        7091
popestimate2019                        7091
abbrv                                  7085
median_household_income_2015           7085
median_household_income_2016           7085
median_household_income_2017           7085
median_household_income_2018           7085
median_household_income_2019           7085
dtype: int64

In [9]:
#keep only columns with 50+ and <50
mortality_df_50 = mortality_df[mortality_df["age"] != "65+"] 
mortality_df_50 = mortality_df_50[mortality_df_50["age"] != "<65"] 
mortality_df_50["age"].value_counts()

50+    2100
<50     993
Name: age, dtype: int64

In [10]:
mortality_df_50.count()

average_annual_count                   3093
primary_state_name                     3093
cancer                                 3093
race_ethnicity                         3093
sex                                    3093
age                                    3093
age_adjusted_rate                      3093
county_state                           3093
index_of_medical_underservice_score    3093
popestimate2015                        3093
popestimate2016                        3093
popestimate2017                        3093
popestimate2018                        3093
popestimate2019                        3093
abbrv                                  3090
median_household_income_2015           3090
median_household_income_2016           3090
median_household_income_2017           3090
median_household_income_2018           3090
median_household_income_2019           3090
dtype: int64

In [11]:
#create target 
y = mortality_df_50[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,38.8
1,606.5
4,30.2
5,626.1
8,27.6


In [12]:
#create feautures variable holding ages a d mua score
X = mortality_df_50[["age", "index_of_medical_underservice_score"]]
X.head()

,age,index_of_medical_underservice_score
0,<50,54.0
1,50+,54.0
4,<50,53.0
5,50+,53.0
8,<50,61.0


In [13]:
#create dummy variables for age
X = pd.get_dummies(X, columns=["age"])
X.head()

,index_of_medical_underservice_score,age_50+,age_<50
0,54.0,0,1
1,54.0,1,0
4,53.0,0,1
5,53.0,1,0
8,61.0,0,1


In [14]:
#get shape of target array
y.shape

(3093, 1)

In [15]:
#get shape of feautures array
X.shape

(3093, 3)

In [16]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
#create linear regression model using sklearn
model = LinearRegression()

In [18]:
#fit the model with training data
regression_age_MUA_mortality = model.fit(X_train, y_train)

In [19]:
#Create predictions array using linear regression
y_pred = regression_age_MUA_mortality.predict(X_test)
y_pred

array([[557.25354298],
       [ 17.93937111],
       [557.13754036],
       [557.13754036],
       [557.10342195],
       [557.11024563],
       [557.15118773],
       [557.11024563],
       [ 17.88478164],
       [557.2057772 ],
       [557.32177982],
       [557.12730484],
       [557.23989562],
       [557.12218708],
       [557.21942457],
       [557.10342195],
       [ 17.87795796],
       [557.03518511],
       [557.19895352],
       [ 17.88478164],
       [557.11195155],
       [557.11024563],
       [ 18.26690792],
       [557.11706931],
       [ 17.85066323],
       [ 17.85748691],
       [557.11706931],
       [557.13754036],
       [ 17.88868089],
       [557.10342195],
       [ 17.87454612],
       [557.16312918],
       [ 17.85066323],
       [557.123893  ],
       [557.10342195],
       [ 17.87113428],
       [ 17.78015183],
       [557.17848247],
       [557.31495613],
       [557.13071668],
       [557.15118773],
       [557.15118773],
       [557.12048116],
       [ 17

In [20]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9200440607212405

In [21]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [22]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [23]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                 1.667e+04
Date:                Wed, 01 Jun 2022   Prob (F-statistic):               0.00
Time:                        20:42:17   Log-Likelihood:                -17813.
No. Observations:                3093   AIC:                         3.563e+04
Df Residuals:                    3090   BIC:                         3.565e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [24]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [25]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([589.4       ,  20.38571429, 562.36162791, 562.36162791,
       560.76333333, 562.50598291, 540.90491803, 562.50598291,
        19.24418605, 590.14482759, 592.53333333, 545.76      ,
       510.44      , 481.6       , 538.42631579, 560.76333333,
        20.14047619, 546.5       , 559.28214286,  19.24418605,
       571.2       , 562.50598291,  15.256     , 561.47281553,
        19.28166667,  17.968     , 561.47281553, 562.36162791,
        16.46666667, 560.76333333,  18.16363636, 579.6       ,
        19.28166667, 559.99310345, 560.76333333,  17.62195122,
        13.6       , 531.42564103, 644.975     , 577.27368421,
       540.90491803, 540.90491803, 531.82727273,  17.13888889,
       561.885     , 559.99310345, 562.36162791, 528.13333333,
        17.968     , 577.27368421,  15.256     , 577.27368421,
       562.50598291, 528.08      , 572.89354839,  17.62195122,
       531.59375   ,  17.968     ,  14.1       , 559.71944444,
       561.47281553,  17.13888889,  16.864     , 585.22

In [26]:
#calculate r2 score
r2_score(y_test, y_pred)

0.9128900084298609